# Import library

In [1]:
import os

In [10]:
from keras.applications import vgg16
from keras.applications import resnet_v2

import numpy as np
from keras.layers import Dense, Dropout, Flatten
from keras import callbacks
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.models import Model 
from IPython.display import clear_output
from keras.models import model_from_json
%matplotlib inline
from keras.utils import multi_gpu_model

In [18]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]

# Split train date to 10 folder

In [12]:
# 1. Make dir
for i in range(0,10):
    path=f'./train/{i}'
    if (os.path.isdir(path)==False):
            os.mkdir(path)

In [13]:
# 2. Split folder
amount=0
for file in os.scandir('./train/'):
    if ( os.path.isdir(file.path)==False and file.path.endswith('.png')):
            fileDir=file.path[-5]
            newPart=f'train/{fileDir}/'
            newPath=file.path.replace('train/',newPart)
            os.rename(file.path,newPath)
            amount+=1
            print(amount)
            if amount%100==0:
                 clear_output()


# Transferlearning vgg16 model 

## Import image by ImageDataGenerators

In [5]:
train_datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest",
        preprocessing_function=vgg16.preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input)

train_generator = train_datagen.flow_from_directory(
    directory=r"./train",
    target_size=(64, 64),
    color_mode="rgb",
    batch_size=200,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    directory=r"./test",
    target_size=(64, 64),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_TEST=test_generator.n

Found 50001 images belonging to 10 classes.
Found 100 images belonging to 1 classes.


## Define Model

In [ ]:
base_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)        # new FC layer, random init
x = Dropout(0.4)(x)
x = Dense(512, activation='relu')(x)        # new FC layer, random init
x = Dropout(0.4)(x)
predictions = Dense(10, activation='softmax')(x)  # 10 labels

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
      layer.trainable = False

## Complie and fit model

In [ ]:
#   Define model compile for basic Transfer Learning
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
model.fit_generator(
                            generator=train_generator,
                            steps_per_epoch=STEP_SIZE_TRAIN,
                            epochs=30
                        )
model_structure=model.to_json()
with open("./vgg16/model.json", "w") as json_file:
    json_file.write(model_structure)
model.save_weights('./vgg16/weight1250.h5')

Epoch 1/30
152/250 [=================>............] - ETA: 4:03 - loss: 0.7118 - accuracy: 0.7777

KeyboardInterrupt: 

## Prediction

In [22]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

100/100 [==============================] - 1s 10ms/step


## Answers

In [23]:
import pandas as pd
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
ans_df = pd.read_csv('./submission.csv')
ans_dict=dict(zip([name[-8:] for name in test_generator.filenames],predictions))
ans_df['label']=ans_df['filename'].apply(lambda name:ans_dict[name])
"".join(ans_df['label'])

'0146762487811409186546468538573847741239027035099830583319889424008410978850048393465030834372470714'

In [ ]:
ans_df.to_csv('./vgg16/submission.csv',index=False)

## Save and load model

In [21]:
json_file = open('./vgg16/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("./vgg16/weight250!!!!.h5")